# Generate clinical plans from patient-physician audio interviews

This notebook demonstrates how to generate clinical plans from patient-physician audio interviews using AWS Managed services and Claude 3 generalised large language model family.  

## Prerequisites
- Verify that model access to Anthropic's Claude 3 Sonnet and Haiku is granted to the account being used, see documentation here: [Amazon Bedrock Model Access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)

## Instructions
1. The notebook is designed to run with Amazon SageMaker Studio. To use Studio, you will need to setup a SageMaker Domain. For instructions on how to onboard to a Sagemaker domain, refer to this [link](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-studio-onboard.html).

2. Update your SageMaker execution role (created when you initially set up the Sagemaker Domain) -- `arn:aws:iam::<AWS_ACCOUNT_ID>:role/service-role/AmazonSageMaker-ExecutionRole-<TIMESTAMP>` -- to
 contain the following IAM policies:

- AmazonBedrockFullAccess
- AmazonTranscribeFullAccess

## Introduction

This notebook shows how to use transcribe and diarize pre-recorded conversations between patients and physicians, and use Claude 3 model family to generate structured clinical notes. 

As shown in the architecture diagram below, this Jupyter Notebook orchestrates:

1. The retrival of patient-physician medical interviews from a public location
2. The upload to the default Sagemaker S3 bucket
3. The execution of an **Amazon Transcribe** batch job to transcribe and diarize the recordings
4. The preparation of the structured prompt to generate the clinical plan
5. Generation of the clinical plan using the Claude 3 model family

![Architecture](assets/clinicalplans_genai.001.png)


## Environment setup

Update boto3 SDK to version **`1.33.0`** or higher.

In [1]:
!pip install botocore boto3 awscli tscribe pandas ipython --upgrade

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 90.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 100.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.0/812.0 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 115.4 MB/s eta 0:00:0000:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ce1e67dffc3dcdaf4749617fae69cc2200604160907bd335190af76cd5835d48
  Stored in directory: /home/sagemaker-user/.cache/pip/w

## 1. Batch Transcription Using Python SDK

Setting up the environment with the AWS clients and libraries

In [19]:
import os
import time
import boto3
import json
import tscribe
import pandas
import datetime
from IPython.display import display_markdown, Markdown, clear_output
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
region = boto3.session.Session().region_name

s3 = boto3.client('s3', region)
transcribe = boto3.client('transcribe', region)
bedrock_runtime = boto3.client('bedrock-runtime', region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'us-east-1'

#### 1.1. Download the recordings

We will use the sample recording published as part of the supplemental materials of the following paper "Fareez, F., Parikh, T., Wavell, C. et al. A dataset of simulated patient-physician medical interviews with a focus on respiratory cases. Sci Data 9, 313 (2022). https://doi.org/10.1038/s41597-022-01423-1 

In [3]:
!curl -L --output data.zip https://springernature.figshare.com/ndownloader/files/30598530

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  986M  100  986M    0     0  39.5M      0  0:00:24  0:00:24 --:--:-- 41.6M


In [4]:
!unzip -qq -o data.zip

In [5]:
prefix = "rawdata"
inputs = sagemaker_session.upload_data(path="Data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-563836490331/rawdata


In the variable below, indicate the name of the recorded session you want to transcribe and summarise:  
- **`[object_name]`**: file name including the extension (e.g. RES0037.mp3)

In [6]:
object_name = "RES0038.mp3"

We will prefill the value of the `[job_name]` variable such to create unique Transcribe jobs.

In [7]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")
media_uri = "s3://%s/%s/%s/%s" % (bucket, prefix, "Audio Recordings", object_name)
job_name = "transcribe-%s-%s" % (object_name.split(".")[0],timestamp)

#### 1.2. Starting an AWS Transcribe job
Invoking **`start_transcription_job`** API to start a transcription job:

In [8]:
response = transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    LanguageCode='en-US',
    Media={
        'MediaFileUri': str(media_uri)
    },
    OutputBucketName=bucket,
    Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2,
        'ChannelIdentification': False
    }
)
print(response)

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-RES0038-2024-03-25-112948', 'TranscriptionJobStatus': 'IN_PROGRESS', 'LanguageCode': 'en-US', 'Media': {'MediaFileUri': 's3://sagemaker-us-east-1-563836490331/rawdata/Audio Recordings/RES0038.mp3'}, 'StartTime': datetime.datetime(2024, 3, 25, 11, 29, 48, 489000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2024, 3, 25, 11, 29, 48, 464000, tzinfo=tzlocal()), 'Settings': {'ShowSpeakerLabels': True, 'MaxSpeakerLabels': 2, 'ChannelIdentification': False}}, 'ResponseMetadata': {'RequestId': 'c71cbc64-84e4-4002-a0ac-a1e8d11a3e73', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c71cbc64-84e4-4002-a0ac-a1e8d11a3e73', 'content-type': 'application/x-amz-json-1.1', 'content-length': '397', 'date': 'Mon, 25 Mar 2024 11:29:48 GMT'}, 'RetryAttempts': 0}}


#### 1.3. Checking job status

The code below will invoke Transcribe **`get_transcription_job`** API to retrieve the status of the job we started in the previous step. If the status is not Completed or Failed, the code waits 5 seconds to retry until the job reaches a final state.

In [9]:
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)

print("Job status: " + status.get('TranscriptionJob').get('TranscriptionJobName'))

start_time = status.get('TranscriptionJob').get('StartTime')
completion_time = status.get('TranscriptionJob').get('CompletionTime')
diff = completion_time - start_time

print("Job duration: " + str(diff))
print("Transcription file: " + status.get('TranscriptionJob').get('Transcript').get('TranscriptFileUri'))

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Job status: transcribe-RES0038-2024-03-25-112948
Job duration: 0:00:51.405000
Transcription file: https://s3.us-east-1.amazonaws.com/sagemaker-us-east-1-563836490331/transcribe-RES0038-2024-03-25-112948.json


#### 1.4. Analysing the scribe results
The code below will download the **`transcribe.json`** file generated by Transcribe, will parse the file and extract the diarised transcription.

In [11]:
transcription_file = job_name + ".json"

transcription = s3.get_object(Bucket=bucket, Key=transcription_file)
body = json.loads(transcription['Body'].read())

s3.download_file(bucket, transcription_file, "output.json")

In [12]:
tscribe.write("output.json", format="csv", save_as="output.csv")

desired_width = 600
pandas.set_option('display.width', desired_width)

transcript = pandas.read_csv("output.csv",  names=["line", "start_time", "end_time", "speaker", "comment"], header=None, skiprows=1)
interaction = ["%s, %s: %s" % (segment[0], segment[1],segment[2]) for segment in transcript[['line','speaker', 'comment']].values.tolist()]
transcript

output.csv written in 1.43 seconds.


,line,start_time,end_time,speaker,comment
0,0,00:00:00,00:00:02,spk_0,So what brings you in here today at the Family...
1,1,00:00:03,00:00:11,spk_1,"Uh, I've been, been coughing these last, uh, t..."
2,2,00:00:11,00:00:13,spk_1,I think I got sick there.
3,3,00:00:14,00:00:18,spk_0,Ok. So just the last couple of weeks you've be...
4,4,00:00:19,00:00:24,spk_1,"Yeah. Ever since I, I got back from Mexico. It..."
...,...,...,...,...,...
146,146,00:11:40,00:11:42,spk_1,"Um Nope, that was it?"
147,147,00:11:43,00:12:00,spk_0,"Ok. So based on what we talked about it, it se..."
148,148,00:12:00,00:12:21,spk_0,um it can also be like a viral upper respirato...
149,149,00:12:22,00:12:24,spk_1,That sounds great. Thank you.


---

## 2. Generate clinical notes using Claude model family

### 2.1. Prompt engineering
Claude is trained to be a helpful, honest, and harmless assistant. It is used to speaking in dialogue, and you can instruct it in regular natural language requests as if you were making requests of a human.The quality of the instructions you give Claude can have a large effect on the quality of its outputs, especially for complex tasks. See https://docs.anthropic.com/claude/docs/intro-to-prompting to learn more about prompt engineering.

Structured enterprise-grade prompts may contain the following sections: 
1. **Task context**
1. Tone context
1. Background data, documents, and images
1. **Detailed task description & rules**
1. Examples
1. Conversation history
1. Immediate task description or request
1. Thinking step by step / take a deep breath

In our scenario, we will use a simplified prompt (template) that will instruct the model to generate a structured summary of the transcribed conversation and indicate the lines in the transcript that support each claim. This summary is divided in the following sections: 

1. Chief complaint
1. History of present illness
1. Review of systems
1. Past medical history
1. Assessment
1. Plan
1. Physical examination

In [13]:
prompt = '''You will be reading a transcript of a recorded conversation between a physician and a patient. You will find the conversation within the transcript XML tags. Your goal is to summarise 
it, capture the most significative insights and propose the appropriate action plan under a section named ‘clinical plan’ that includes the following sections: Chief complaint; History of present 
illness; Review of systems; Past medical history; Assessment; Plan; Physical examination. Per each claim you make, you need to indicate which lines of the transcript supports it (please indicate 
only the line numbers within the tag <line></line>).
<transcript>
%s
</transcript>
''' % "\n".join(interaction)
print(prompt)

You will be reading a transcript of a recorded conversation between a physician and a patient. You will find the conversation within the transcript XML tags. Your goal is to summarise 
it, capture the most significative insights and propose the appropriate action plan under a section named ‘clinical plan’ that includes the following sections: Chief complaint; History of present 
illness; Review of systems; Past medical history; Assessment; Plan; Physical examination. Per each claim you make, you need to indicate which lines of the transcript supports it (please indicate 
only the line numbers within the tag <line></line>).
<transcript>
0, spk_0: So what brings you in here today at the Family Clinic?
1, spk_1: Uh, I've been, been coughing these last, uh, two weeks since I got back from, uh, Mexico.
2, spk_1: I think I got sick there.
3, spk_0: Ok. So just the last couple of weeks you've been coughing?
4, spk_1: Yeah. Ever since I, I got back from Mexico. It's been, yeah, so about, about

### 2.2. Payload preparation and model invocation
The new generation of Claude model only support the Messages API, hence we must format the body of our payload in the following way:

In [14]:
accept = 'application/json'
contentType = 'application/json'
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1000,
        "messages": [
            {
                "role": "user",
                "content": [{
                    "type": "text",
                    "text": prompt,
                }],
            },
        ],
        "temperature": 0
    }
)

#### 2.2.1 Claude 3 Sonnet

The Bedrock service generates the entire summary for the given prompt in a single output, this can be slow if the output contains large amount of tokens.

Below we explore the option how we can use Bedrock to stream the output such that the user could start consuming it as it is being generated by the model. For this Bedrock supports invoke_model_with_response_stream API providing ResponseStream that streams the output in form of chunks.

Instead of generating the entire output, Bedrock sends smaller chunks from the model. This can be displayed in a consumable manner as well.


In [15]:
def teletype_model_response(stream):
    output = []
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                if chunk_obj['type'] == 'content_block_delta':
                    text = chunk_obj['delta']['text']
                    clear_output(wait=True)
                    output.append(text)
                    display_markdown(Markdown(''.join(output)))
                    i += 1

We will print the content of the response immediately as the first string is returned 

In [16]:
%%time
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'

# response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
# response_body = json.loads(response["body"].read())
# completion = response_body["content"][0]["text"]
# print(completion)
response = bedrock_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
teletype_model_response(response.get('body'))


Clinical Plan:

Chief Complaint:
- Persistent dry cough for the past 2 weeks since returning from Mexico (<line>1</line>, <line>4</line>)

History of Present Illness:
- Dry cough without sputum production (<line>6</line>)
- No coughing up blood (<line>8</line>)
- Cough comes and goes, worsens with exercise, cold weather (<line>12</line>, <line>13</line>, <line>14</line>)
- Cough not getting worse (<line>19</line>)
- No cold symptoms like runny nose or sore throat (<line>27</line>)
- Cough affects sleep and causes coughing fits at night (<line>35</line>)
- Cough relieved by using son's inhaler (<line>38</line>)
- Occasional wheezing noted (<line>103</line>)

Review of Systems:
- No headache, nausea, or vomiting (<line>48</line>)
- No fever, chills, or night sweats (<line>62</line>)
- No dizziness or palpitations (<line>69</line>, <line>71</line>)
- No chest pain (<line>73</line>)
- No changes in bowel movements or urinary patterns (<line>75</line>, <line>77</line>)
- No weight changes or appetite changes (<line>82</line>, <line>84</line>)
- No loss of taste or smell (<line>88</line>)
- Occasional shortness of breath and itchy eyes with outdoor exposure (<line>53</line>, <line>54</line>)

Past Medical History:
- History of eczema in childhood (<line>58</line>, <line>59</line>)
- Possible remote history of asthma in childhood (<line>43</line>)
- No hospitalizations or surgeries (<line>107</line>, <line>109</line>)

Assessment:
- Possible asthma exacerbation or reactive airway disease
- Potential environmental or allergic triggers (cats, pollen, outdoor exposures)
- Recent smoking cessation (within the last month)
- Recreational marijuana and alcohol use

Plan:
- Perform pulmonary function tests
- Trial of bronchodilator therapy (inhaler)
- Evaluate for environmental triggers (cats, outdoor allergens)
- Counsel on smoking cessation and substance use
- Consider allergy testing if symptoms persist

Physical Examination:
(No details provided in the transcript)

CPU times: user 1.34 s, sys: 337 ms, total: 1.68 s
Wall time: 15.5 s


#### 2.2.2. Claude 3 Haiku

Let's print the response only when it is returned in full

In [17]:
%%time
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'
response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response["body"].read())
completion = response_body["content"][0]["text"]
display_markdown(Markdown(''.join(completion)))

Clinical Plan:

Chief Complaint: Persistent dry cough for the past 2 weeks since returning from Mexico.

History of Present Illness:
- The patient has been experiencing a dry cough for the past 2 weeks since returning from a trip to Mexico (<line>1</line>, <line>2</line>, <line>4</line>).
- The cough is variable in nature, sometimes worse with exercise or cold weather (<line>13</line>, <line>14</line>, <line>15</line>).
- The cough is worse at night and is impacting the patient's sleep (<line>35</line>, <line>50</line>).
- The patient has tried using an inhaler belonging to their son, which seemed to help with the cough (<line>38</line>).

Review of Systems:
- The patient denies any other symptoms such as runny nose, sore throat, fever, chills, night sweats, chest pain, or changes in appetite or weight (<line>24</line>, <line>25</line>, <line>45</line>, <line>47</line>, <line>61</line>, <line>62</line>, <line>63</line>, <line>81</line>, <line>83</line>).
- The patient reports occasional wheezing and shortness of breath, particularly with exertion (<line>53</line>, <line>54</line>, <line>102</line>, <line>103</line>).
- The patient also reports a history of eczema in the past (<line>58</line>, <line>59</line>).

Past Medical History:
- The patient had a remote history of asthma in childhood, but has not had any symptoms for several years (<line>29</line>, <line>30</line>, <line>41</line>, <line>43</line>, <line>44</line>).
- The patient has no other significant medical conditions (<line>94</line>, <line>95</line>).

Assessment:
- The patient's symptoms, including the dry cough, wheezing, and shortness of breath, are suggestive of possible asthma or a viral upper respiratory tract infection.
- The patient's history of eczema and past asthma may also be contributing factors.
- The patient's use of an inhaler and the apparent improvement in symptoms provide further support for an asthmatic component.

Plan:
1. Perform pulmonary function tests to assess for asthma (<line>148</line>).
2. Recommend a trial of a bronchodilator medication, such as the patient's son's inhaler, to see if it provides symptomatic relief (<line>148</line>).
3. Advise the patient to avoid potential triggers, such as exercise and cold weather, that may exacerbate the cough (<line>13</line>, <line>14</line>, <line>15</line>).
4. Discuss the patient's history of smoking and provide support for continued smoking cessation efforts (<line>132</line>, <line>133</line>).
5. Recommend the patient limit alcohol consumption and recreational drug use, as these may be contributing to the cough (<line>136</line>, <line>139</line>, <line>141</line>).
6. Encourage the patient to maintain a healthy lifestyle, including regular exercise and a balanced diet (<line>142</line>, <line>143</line>).
7. Schedule a follow-up appointment to monitor the patient's progress and adjust the treatment plan as needed.

Physical Examination:
A comprehensive physical examination should be performed, focusing on the respiratory system and any signs of allergic or inflammatory conditions.

CPU times: user 6.59 ms, sys: 0 ns, total: 6.59 ms
Wall time: 8.38 s
